In [1]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import locale
import os

### Comuni ISTAT 2020

In [2]:
gdf_comuni = gpd.read_file(r"../input/shp/centroidi_comuni_bdn.shp")[['ISTAT','geometry']]
gdf_comuni.rename(columns={'ISTAT':'CodIstat'}, inplace=True)
gdf_comuni["CodIstat"] = gdf_comuni["CodIstat"].astype(int)
# gdf_comuni.head(5)

### Focolai puliti

importa i focolai già ripuliti dal precedente script

In [3]:
df_focolai = pd.read_excel(r"../output/focolai_wn.xlsx")
# df_focolai.head(3)

### Aggregazione (dal 1/1/2008 al 31/12/2018)

In [4]:
df_focolai = df_focolai[["DataSospetto","IdFocolaio","CodIstat","Specie","Categorie","Sierotipo","Regione","Prov","Comune"]].query("DataSospetto >= '2008-01-01' & DataSospetto <= '2018-12-31'")
df_aggregato = df_focolai.groupby(['DataSospetto','CodIstat','Categorie','Regione','Prov','Comune'])["CodIstat"].count().reset_index(name="count")
distribuzione = gdf_comuni.merge(df_aggregato, on='CodIstat')

### Esportazione

In [5]:
distribuzione.to_file(r"../output/distr_wn_centroidi.geojson", driver='GeoJSON')

In [6]:
distribuzione.head(5)

,CodIstat,geometry,DataSospetto,Categorie,Regione,Prov,Comune,count
0,23010,POINT (11.36684 45.20762),2009-11-04,EQUIDI,VENETO,VR,BOSCHI SANT'ANNA,1
1,6015,POINT (8.44978 44.82829),2016-09-27,UCCELLI TARGET,PIEMONTE,AL,BERGAMASCO,1
2,38005,POINT (12.12810 44.85816),2008-09-15,AVICOLI,EMILIA ROMAGNA,FE,CODIGORO,1
3,38005,POINT (12.12810 44.85816),2008-09-23,EQUIDI,EMILIA ROMAGNA,FE,CODIGORO,1
4,38005,POINT (12.12810 44.85816),2008-09-29,EQUIDI,EMILIA ROMAGNA,FE,CODIGORO,1
